In [2]:
import csv
import pandas as pd

In [3]:
hostipitality_customers = pd.read_csv('data/HospitalityCustomers.txt')

c:\users\tkyipicard\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [50]:
reservations = pd.read_csv('data/Reservations.txt')

In [52]:
restaurant_locations = pd.read_csv('data/Restaurant_Locations.txt')

In [6]:
survey_contacts = pd.read_csv('data/SurveyContacts.txt')

c:\users\tkyipicard\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
surveys = pd.read_csv('data/Surveys.txt')

## delete all the empty columns

In [22]:
surveys=surveys.drop(columns=['AttributeA', 'AttributeB', 'AttributeC', 'AttributeD', 'AttributeE'])

In [21]:
survey_contacts=survey_contacts.drop(columns=['AttributeA', 'AttributeB', 'AttributeC', 'AttributeD'])

In [53]:
restaurant_locations=restaurant_locations.drop(columns=['AttributeA', 'AttributeB', 'AttributeC', 'AttributeD'])

In [54]:
reservations=reservations.drop(columns=['AttributeA', 'AttributeB', 'AttributeC', 'AttributeD', 'AttributeE'])

In [17]:
hostipitality_customers = hostipitality_customers.drop(columns=['AttributeA', 'AttributeB', 'AttributeC', 'AttributeD'])

In [26]:
# merge all the dataframes containing CustomerId
hostipitality_customers.CustomerID.value_counts()

A0DA01E5-B48D-4996-AA45-9E33D8D71329    1
3B1F0A81-94F3-4966-8B30-392ACC3BB555    1
2A8DC565-6F28-4F41-8AE5-DEC5912AD31E    1
0D55E393-F7A1-46D9-BB25-A36D88738141    1
4DAD04AB-6136-434F-9868-40635CA7D081    1
                                       ..
4DE2DBDA-F903-4939-863F-FC709B9AB832    1
DBFD0A5A-F824-4731-9B40-4458E4D8F8EA    1
5F3CAFB1-F5E7-4E4C-AB0C-62173DEB70E7    1
599EA6E1-C46E-4441-A6BC-49C6D1001FB6    1
44C81195-29FB-46A5-A2F9-BCEE0AD18CB0    1
Name: CustomerID, Length: 1000006, dtype: int64

In [43]:
reservations.LocationID.value_counts()

6C274696-5B7B-48F3-AB40-B0FBFB4C26CE    107893
9F0B1BA9-4A0B-4280-9FB4-76DDDD2122CC    107654
7F8F11EF-A188-442A-8FBB-CC53B033B260     82442
3A7988CF-E05A-4668-B972-244D92815B87     47914
7AEF27BE-7CA1-4DAB-BE41-C39EE9C626B0     37501
                                         ...  
B4ADD95D-565F-47F8-9272-DF5193402CFC       724
8009F4BA-D30A-48A4-AD32-E5B4C9EDE006       712
EBF86331-82DC-49E4-A462-9C349733BC06       698
5F6D8127-E2E4-45BE-B23C-C1EA26635BDB       693
AE179A2E-E250-445A-8F3B-2648E69CDF0F       683
Name: LocationID, Length: 725, dtype: int64

In [27]:
reservations.CustomerID.value_counts()

450C0282-4AE4-4467-8C4F-B088C401B2AE    88
08091E8D-EF9F-43F1-A77E-C89F0B70404C    75
EB288AEA-AB75-4715-994A-2F870683A881    73
B27FE399-D5BA-4625-A6B6-CD2809B935FC    68
825CF2C3-518B-4461-A112-160785857118    61
                                        ..
A713D214-CA7F-47AA-BA4B-C0598D1B4E71     1
853C034E-20EB-48DD-B2EE-1A684EDA5C67     1
581B8E3C-9969-4509-B942-F25DA30123C6     1
BDB4626E-2C93-4BC0-9C60-1CDF99EE8791     1
44C81195-29FB-46A5-A2F9-BCEE0AD18CB0     1
Name: CustomerID, Length: 1000006, dtype: int64

In [33]:
restaurant_locations.LocationID.value_counts()

B6B802EC-5783-4C32-A6A8-AD90D1AB2C36    1
08073CAA-B50B-4F3C-BDDE-7B25669DD1F4    1
1ACC3C04-E5F5-4CD0-A670-C406C9214966    1
E815BDDC-8062-41AC-B46F-D39D6831EFD6    1
B8C5B455-26B5-4BB9-90CC-1CB0CA10B168    1
                                       ..
C5779AAA-32BC-48AC-BAC6-C3682CC2FC00    1
04F487F9-5199-4DED-AE5A-584D3EE1AE68    1
7745CA5E-6D7B-4658-B340-654643482C4D    1
F5FEC172-8679-4E14-9B1D-80AD4DF51BBD    1
DEF79348-1381-47F8-9825-9342FEBAE770    1
Name: LocationID, Length: 931, dtype: int64

In [35]:
surveys.SurveyID.value_counts()

CA5EE322-185B-4B8B-BF77-460E927D8EA7    1
01E4A8F4-06D0-4EC3-9260-3FB971B61C80    1
792F6ACC-34AC-4D61-AFE9-CB18A21C4EB3    1
38004ABD-2D87-4A34-9728-958550D5294C    1
089A3320-815E-4BEC-975B-1AB66CE9C3C5    1
                                       ..
12BFB652-28EF-4F9B-808F-F47E3906538A    1
53534D1A-BE72-4CE3-83BF-28D48D53649C    1
2D1FBF24-7667-4C1D-91E6-43754B760269    1
8A8C16AF-F753-486D-A2AB-7A483AFCEDE2    1
0BF0B6C4-7FC4-487D-9FEA-DEB8842AE346    1
Name: SurveyID, Length: 1149882, dtype: int64

In [37]:
surveys.SurveyContactID.value_counts()

3A6CD05E-73B1-4393-945B-B4C2310F64C0    52
CF3A1F6C-3E66-4A6B-BD98-6C8DE9DA66DD    49
9845CD8E-3060-4D7E-903E-F1587DF893F9    47
35FF569B-863D-4C59-9045-D5A536F20EF9    44
49D5D381-2AD2-4C98-9EBE-B01C1F9BA461    42
                                        ..
A69C4CDB-2451-46AF-91CE-90856790E194     1
480A7010-BE96-45E2-94AB-EDA74D3953E8     1
0AC835C4-2EC8-4F82-BBA9-0B9FA2F0AA8F     1
08306F35-0700-4540-A7CE-8879862E1F86     1
E44F0E93-4A03-47D4-AF33-02B88DB7947A     1
Name: SurveyContactID, Length: 748877, dtype: int64

In [39]:
survey_contacts.SurveyContactID.value_counts()

8C597FC9-747B-4A8A-AAB4-C57495D3D7A0    1
D81BE5B9-0B44-45B4-BB02-6FE628BC457C    1
10D7828E-1D1F-4D7F-8D59-77560BD27F00    1
C50DBA52-20DA-4132-A91D-86A05F182B0C    1
CE926273-235E-47D2-BEEE-B5A34CE029EE    1
                                       ..
65F370E2-824C-418B-B572-AB78E5E3BD64    1
C7E4682B-5D54-40F0-9E18-A86D03E66472    1
31E842F6-9E3D-4594-A01C-AD1767AECB4C    1
D43136E8-B0CE-499C-8356-5744C99CED38    1
CC838E7F-5AFE-4CFA-828D-86719690E9B7    1
Name: SurveyContactID, Length: 748877, dtype: int64

## Combining dataframes

In [82]:
# merge reservations and hospitality_customers with CustomerID. Both have same number of lines
hospitality_reservations = hostipitality_customers.merge(reservations, how='inner', on='CustomerID')

In [83]:
# locationID de hospitality_reservations avec restaurant_locations
hos_res_loc = hospitality_reservations.merge(restaurant_locations, how='inner', on="LocationID")

In [115]:
#merging surveys and surveys_contacts
surveys_survey_contacts = survey_contacts.merge(surveys, how='inner', on='SurveyContactID')

In [85]:
#rename columns
import re
hos_res_loc = hos_res_loc.rename(columns=lambda x: re.sub('_x','',x))
hos_res_loc = hos_res_loc.rename(columns=lambda y: re.sub('_y', '_restaurant', y))

In [87]:
#1566569 rows × 31 columns without dropna
#418 rows × 31 columns with dropna

In [99]:
names = hos_res_loc.Name.unique()
print(len(names))

942269


In [100]:
res_names = hos_res_loc.Name_restaurant.unique()
print(len(res_names))

343


In [118]:
hos_res_loc.columns

Index(['Name', 'Address', 'City', 'State', 'Country', 'Zip', 'EmailId',
       'Gender', 'DOB', 'Phone', 'SSN', 'Individual', 'ExtraActivity',
       'CustomerID', 'ReservationID', 'LocationID', 'Source',
       'ReservationDate', 'CheckOutDate', 'NumberOfGuests', 'TotalPrice',
       'Subject', 'Description', 'Name_restaurant', 'Address_restaurant',
       'City_restaurant', 'State_restaurant', 'Zip_restaurant',
       'Country_restaurant', 'PhoneNumber', 'NoOfTables'],
      dtype='object')

In [126]:
surveys_survey_contacts.columns

Index(['SurveyContactID', 'Name', 'Email', 'DOB', 'Gender', 'StreetAddress',
       'City', 'StateProvince', 'Country', 'PostalCode', 'PhoneNumber',
       'ExtraActivity', 'SurveyID', 'SurveyDate', 'SurveyQuestion', 'Response',
       'Subject', 'Description'],
      dtype='object')

In [128]:
#check the number of phone number matches between the two columns
hos_res_loc['PhoneNumber'].isin(hos_res_loc['Phone']).value_counts()#

False    1512944
True       53625
Name: PhoneNumber, dtype: int64

In [135]:
#check the number of phone number matches between the two columns
hos_res_loc['Phone'].isin(hos_res_loc['PhoneNumber']).value_counts()#

False    1489065
True       77504
Name: Phone, dtype: int64

## comparing surveys_survey_contacts dataframes with hos_res_loc dataframe

In [130]:
#Comparing column names in two different dataframes
set(hos_res_loc.columns).intersection(set(surveys_survey_contacts.columns))

{'City',
 'Country',
 'DOB',
 'Description',
 'ExtraActivity',
 'Gender',
 'Name',
 'PhoneNumber',
 'Subject'}

In [134]:
#Comparing values in two different columns of two different dataframes
set(hos_res_loc.CustomerID).intersection(set(surveys_survey_contacts.SurveyContactID))

set()